In [1]:
import os
import fastavro
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from correction.compute import *

In [2]:
from alerce.api import AlerceAPI
client = AlerceAPI()

In [3]:
AVRO_PATH = "../tests/data_examples/"

# Functions to manipulate example data

In [4]:
def read_avro(avro_path):
    with open(avro_path, "rb") as file:
        try:
            reader = fastavro.reader(file)
            data = reader.next()
        except Exception as e:
           return None
    return data

In [5]:
def get_avros(oid):
    path = os.path.join(AVRO_PATH, oid)
    avros = sorted(os.listdir(path))
    data = []
    for a in avros:
        content = read_avro(os.path.join(path, a))
        if content:
            data.append(content)
    return data

In [6]:
detection_columns = ['oid', 'candid', 'jd', 'fid', 'pid', 'diffmaglim', 'isdiffpos', 'nid', 'ra', 'dec', 'magpsf', 'sigmapsf', 'magap', 'sigmagap', 'distnr', 'rb', 'rbversion', 'drb', 'drbversion', 'magapbig', 'sigmagapbig', 'rfid', 'magpsf_corr', 'sigmapsf_corr', 'sigmapsf_corr_ref', 'has_stamp', 'corr', 'flag', 'candid_alert']
non_detection_columns = ['oid', 'jd', 'fid', 'diffmaglim']
object_columns = ['oid', 'ndethist', 'ncovhist', 'jdstarthist', 'jdendhist', 'stellar', 'ndet', 'meanra', 'meandec', 'sigmara', 'sigmadec', 'deltajd', 'firstmjd', 'lastmjd']
magstats_columns = ['oid', 'rfid', 'fid', 'magnr', 'nrfid', 'stellar', 'corr', 'ndet', 'dmdt', 'magmean', 'magmedian', 'magmax', 'magmin', 'magsigma', 'maglast', 'magfirst', 'firstmjd', 'lastmjd', 'step_id_corr', 'step_id_feat']


In [7]:
# This function must be implement in step, but querying to db
def is_first_detection(df, alert):
    candid = alert["candid"]
    oid = alert["objectId"]
    fid = alert["candidate"]["fid"]
    mask = (df["oid"] == oid) & (df["fid"] == fid)
    table = df[mask]
    return len(table) == 0

def get_first_magnr(df, data):
    oid = data["objectId"]
    candid = data["candidate"]["fid"]
    mask = (df["oid"] == oid) & (df["fid"] == fid)
    return df[mask]["magnr"].values[0]

def insert_detection(df, alert, magpsf_corr, sigmapsf_corr, sigmapsf_corr_ref, flag, has_stamp, corr):
    data = {}
    for k in alert["candidate"]:
        if k in detection_columns:
            data.update({k: alert["candidate"][k]})
    data.update({
        "oid": alert["objectId"],
        "magpsf_corr": magpsf_corr, 
        "sigmapsf_corr": sigmapsf_corr, 
        "sigmapsf_corr_ref": sigmapsf_corr_ref, 
        "flag": flag, 
        "has_stamp": has_stamp, 
        "corr": corr
    })
    return df.append([data])


def insert_non_detection(df, oid, jd, fid, diffmaglim):
    data = {
        "oid": oid,
        "jd": jd,
        "fid": fid,
        "diffmaglim": diffmaglim
    }
    return df.append([data])

def insert_object(df, alert, stellar):
    data = {}
    for k in alert["candidate"]:
        if k in object_columns:
            data.update({k: alert["candidate"][k]})
    data.update({
        "oid": alert["objectId"],
        "stellar": stellar
    })
    return df.append([data])

def insert_magstats(df, alert):
    data = {}
    for k in alert["candidate"]:
        if k in magstats_columns:
            data.update({k: alert["candidate"][k]})
    data.update({
        "oid": alert["objectId"],
    })
    return df.append([data])

In [8]:
oids = os.listdir(AVRO_PATH)
example = get_avros(oids[0])

In [10]:
detection_table = pd.DataFrame(columns=detection_columns)
non_detection_table = pd.DataFrame(columns=non_detection_columns)
object_table = pd.DataFrame(columns=object_columns)
magstats_table = pd.DataFrame(columns=magstats_columns)

In [11]:
for idx, data in enumerate(example):
    is_first = is_first_detection(detection_table, data)
    has_stamp = True
    fid = data["candidate"]["fid"]    
    candid =  data["candid"]
    
    if is_first:
        magstats_table = insert_magstats(magstats_table, data)
    
    # object
    stellar_object, stellar_magstats = validate_object(data["candidate"], is_first)
    corr_detection, corr_magstats, flag = validate_magnitudes(data["candidate"])
    magpsf_corr, sigmapsf_corr, sigmapsf_corr_ref = apply_correction(data["candidate"])
    
    detection_table = insert_detection(detection_table, data,  magpsf_corr, sigmapsf_corr, sigmapsf_corr_ref, flag, has_stamp, corr_detection)

In [12]:
magstats_table

,oid,rfid,fid,magnr,nrfid,stellar,corr,ndet,dmdt,magmean,magmedian,magmax,magmin,magsigma,maglast,magfirst,firstmjd,lastmjd,step_id_corr,step_id_feat
0,ZTF18aazxcwf,481120101,1,15.888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ZTF18aazxcwf,481120201,2,15.716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
detection_table

,oid,candid,jd,fid,pid,diffmaglim,isdiffpos,nid,ra,dec,...,magapbig,sigmagapbig,rfid,magpsf_corr,sigmapsf_corr,sigmapsf_corr_ref,has_stamp,corr,flag,candid_alert
0,ZTF18aazxcwf,1007101110115010003,2.458762e+06,1,1007101110115,19.239826,f,1007,242.777292,2.055899,...,16.906200,0.0595,481120101,16.387620,100.000000,0.026419,True,True,False,NaN
0,ZTF18aazxcwf,1007109840115010001,2.458762e+06,2,1007109840115,19.545088,f,1007,242.777326,2.055944,...,17.180201,0.0623,481120201,16.064998,0.015829,0.023916,True,True,False,NaN
0,ZTF18aazxcwf,1097547910115010011,2.458852e+06,1,1097547910115,18.471132,f,1097,242.777135,2.055992,...,17.472500,0.1226,481120101,16.172542,0.031314,0.040696,True,True,False,NaN
0,ZTF18aazxcwf,1100569180115010020,2.458855e+06,2,1100569180115,19.031256,f,1100,242.777217,2.055934,...,16.906799,0.0355,481120201,16.193164,0.021334,0.029363,True,True,False,NaN
0,ZTF18aazxcwf,1122541220115010018,2.458877e+06,1,1122541220115,20.512083,f,1122,242.777308,2.055893,...,16.932199,0.0217,481120101,16.363024,0.017249,0.035456,True,True,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,ZTF18aazxcwf,972136730115010008,2.458727e+06,2,972136730115,19.893042,f,972,242.777325,2.055924,...,16.771400,0.0327,481120201,16.223546,0.025399,0.032796,True,True,False,NaN
0,ZTF18aazxcwf,972166510115010007,2.458727e+06,1,972166510115,20.286003,f,972,242.777319,2.055913,...,16.803301,0.0234,481120101,16.478261,0.018842,0.039262,True,True,False,NaN
0,ZTF18aazxcwf,980169140115010006,2.458735e+06,1,980169140115,19.388327,f,980,242.777298,2.055933,...,17.198000,0.0601,481120101,16.276152,100.000000,0.021342,True,True,False,NaN
0,ZTF18aazxcwf,983172500115010008,2.458738e+06,2,983172500115,19.167662,f,983,242.777311,2.055922,...,16.971701,0.0561,481120201,16.134022,0.017716,0.026055,True,True,False,NaN


In [14]:
detection_table.columns

Index(['oid', 'candid', 'jd', 'fid', 'pid', 'diffmaglim', 'isdiffpos', 'nid',
       'ra', 'dec', 'magpsf', 'sigmapsf', 'magap', 'sigmagap', 'distnr', 'rb',
       'rbversion', 'drb', 'drbversion', 'magapbig', 'sigmagapbig', 'rfid',
       'magpsf_corr', 'sigmapsf_corr', 'sigmapsf_corr_ref', 'has_stamp',
       'corr', 'flag', 'candid_alert'],
      dtype='object')